# Gaze Analyser
This notebook takes in three files taken from the Pupil Recordings you have exported in Pupil Player. These are:
- `gaze_positions.csv` (contains raw data in regards to the gaze made throughout the recording)
- `info.player.json` (contains system and sync time used to format the recording timestamps)
- `fixations.csv` (contains events of where fixations has occured throughout the experiment)
- `annotations.csv` (contains annotations created in the experiment used for object interception/spawn, regions of observation, and when the experiment has begin and ended)

In [117]:
# Imports
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import warnings

matplotlib.use('TkAgg')
warnings.filterwarnings("ignore", category=RuntimeWarning)

# File Paths
info_player_filePath = 'experiment_source/info.player.json'
gaze_csv_filePath = 'experiment_source/gaze_positions.csv'
annotations_filepath = 'experiment_source/annotations.csv'
fixation_filepath = 'experiment_source/fixations.csv'

# Contrasting Colours
CONTRASTING_COLURS = ['#011627', '#2ec4b6', '#e71d36', '#ff9f1c']
current_colour_index = 0

## Creating the DataFrame from the data

Reading the `info.player` JSON to retrieve `start_time_synced_s` and `start_time_system_s`. This is used to format the timestamp correctly to indicate time throughout the experiment:

In [118]:
def obtain_offset(filepath):
    with open(info_player_filePath, 'r') as file:
        data = json.load(file)  
    return data.get('start_time_system_s') - data.get('start_time_synced_s')

Now, reading the gaze_position.csv file to obtain base data, formating time, calculating smoothed positions, and angular distance and velocity of each timestamp. And removing data that contains infinte or NaN values.

In [119]:
# Calculate angular distance given Cartesian coordinates (x, y)
def calculate_angular_distance(x, y):
    return np.arctan2(y, x)

# Calculate velocity given angular distance and corresponding timestamps.
def calculate_velocity(angular_distance, timestamp):
    time_diff = np.diff(timestamp)
    angular_distance_diff = np.diff(angular_distance)
    velocity = angular_distance_diff / time_diff
    return np.concatenate(([np.nan], velocity))


# Applies Rolling Medium over the input field, and exports that into the output field
def smooth_data(df, input, output, window_size=0):
    df[output] = df[input].rolling(window=window_size).median()
    return df

def obtain_gaze_data(filepath: str, window=10, offset=0):
    csv_df = pd.read_csv(filepath)
    results_df = pd.DataFrame(columns=['gaze_timestamp',
                                       'time',
                                       'norm_pos_x', 
                                       'norm_pos_y', 
                                       'angular_distance',
                                       'angular_velocity',
                                       'movement_type',
                                       'smoothed_norm_pos_x', 
                                       'smoothed_norm_pos_y',
                                       'smoothed_angular_distance',
                                       'smoothed_angular_velocity',
                                       'smoothed_movement_type'])

    # Copy the data from csv of necessary fields
    results_df['gaze_timestamp'] = csv_df['gaze_timestamp']
    results_df['norm_pos_x'] = csv_df['norm_pos_x']
    results_df['norm_pos_y'] = csv_df['norm_pos_y']
    results_df['movement_type'] = 'None'
    results_df['smoothed_movement_type'] = 'None'
    
    # Populating the time field by adding the offset and then subtracting the minimum time to start from 0
    results_df['time'] = results_df['gaze_timestamp'] + offset
    results_df['time'] -= results_df['time'].min()

    # Using rolling mean to smooth the data and to rmeove as many extreme outliers that has been missed by Pupil Export
    results_df['smoothed_norm_pos_x'] = results_df['norm_pos_x'].rolling(window).median()
    results_df['smoothed_norm_pos_y'] = results_df['norm_pos_y'].rolling(window).median()

    # Calculating the angular distance for each x and y position for both smoothed and un-smoothed
    results_df['angular_distance'] = calculate_angular_distance(results_df['norm_pos_x'], results_df['norm_pos_y'])
    results_df['smoothed_angular_distance'] = calculate_angular_distance(results_df['smoothed_norm_pos_x'], results_df['smoothed_norm_pos_y'])

    # Calculating the angular velocity for both smooth and un-smoothed angular distances over time
    results_df['angular_velocity'] = calculate_velocity(results_df['angular_distance'], results_df['time'])
    results_df['smoothed_angular_velocity'] = calculate_velocity(results_df['smoothed_angular_distance'], results_df['time'])

    # Dropping NaN records and the `gaze_timestamp` field
    results_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    results_df.dropna(inplace=True)
    results_df = results_df.drop('gaze_timestamp', axis=1) # Remove the `gaze_timestamp` as that is not needed anymore

    return results_df

window_size=10
offset = obtain_offset(info_player_filePath)
gaze_df = obtain_gaze_data(gaze_csv_filePath, window=window_size, offset=offset)
print(f'There is a total of {len(gaze_df)} gaze positions')


There is a total of 7629 gaze positions


### Predicting Smooth Pursuit
Smooth pursuit occurs when the eyes tracks an moving object. There is roughly a constant velocity as the angular distance changes slowly, whereas Saccades are almost instant. We choose regions of the line where the change in velocity is under a threshold, and the change of angular distance is small enough to indicate the eyes moving between positions slowly.

In [120]:
def predict_smooth_pursuit(df, threshold_velocity=0.2, angular_distance_threshold=2e-4):
    pursuit_regions_velocity = np.abs(np.gradient(df['angular_velocity'])) < threshold_velocity
    pursuit_regions_distance = np.abs(np.gradient(df['angular_distance'])) > angular_distance_threshold
    pursuit_regions = (pursuit_regions_velocity) & (pursuit_regions_distance)
    df['movement_type'] = np.where((df['movement_type'] == 'None') & pursuit_regions, 'Smooth Pursuit', df['movement_type'])
    
    pursuit_regions_smoothed_velocity = np.abs(np.gradient(df['smoothed_angular_velocity'])) < threshold_velocity
    pursuit_regions_smoothed_distance = np.abs(np.gradient(df['smoothed_angular_distance'])) > angular_distance_threshold
    pursuit_regions_smoothed = (pursuit_regions_smoothed_velocity) & (pursuit_regions_smoothed_distance)
    df['smoothed_movement_type'] = np.where((df['smoothed_movement_type'] == 'None') & pursuit_regions_smoothed, 'Smooth Pursuit', df['smoothed_movement_type'])

predict_smooth_pursuit(gaze_df, threshold_velocity=0.2, angular_distance_threshold=2e-4)
smooth_pursuit = gaze_df[(gaze_df['movement_type'] == 'Smooth Pursuit') | (gaze_df['smoothed_movement_type'] == 'Smooth Pursuit')]
print(f'Total Smooth Pursuit Movements: {len(smooth_pursuit)}')

Total Smooth Pursuit Movements: 5641


### Predicting Saccades
Based on the research from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1190820/pdf/jphysiol00502-0164.pdf, Saccadic Movements have a stereotypical velocity graph where large peaks indicates a fast movement between two points (saccades). These are almost like straight lines between points. We can say that if the velocity between two  points exceeds a threshold, then this is most possibly a saccadic movement.

In [121]:
def predict_saccades(df, threshold_velocity=0.6):
    saccade_regions_high = df['angular_velocity'] > threshold_velocity
    accade_regions_low = df['angular_velocity'] < -threshold_velocity
    saccade_regions = saccade_regions_high | accade_regions_low
    df['movement_type'] = np.where(saccade_regions, 'Saccades', 'None')
    
    saccade_regions_high = df['smoothed_angular_velocity'] > threshold_velocity
    accade_regions_low = df['smoothed_angular_velocity'] < -threshold_velocity
    saccade_regions = saccade_regions_high | accade_regions_low
    df['smoothed_movement_type'] = np.where((df['smoothed_movement_type'].isin(['None', 'Smooth Pursuit'])) & saccade_regions, 'Saccades', df['smoothed_movement_type'])

predict_saccades(gaze_df, threshold_velocity=0.6)
saccades = gaze_df[(gaze_df['movement_type'] == 'Saccades') | (gaze_df['smoothed_movement_type'] == 'Saccades')]
print(f'Total Saccadic Movements: {len(saccades)}')

Total Saccadic Movements: 2367


### Predicting Fixation
Fixations are defined as gaze remain fixed around a stationary point. It is not directly fixed but the user gazes around the same point for a long period of time.

Pupil Player provides another csv file called `fixations` which identifies the timestamp of when a fixation occurs, and how long it can take. The function `plot_fixation_graph` takes regions starting from whatever second the fixation starts up until the end of the fixation indicated by by the `duration` field.

In [122]:
def predict_fixation(df, offset, time_offset=0.10, filepath='fixations.csv'):
    fixations_df = pd.read_csv(filepath)
    fixations_df['time'] = fixations_df['start_timestamp'] + offset
    fixations_df['time'] -= fixations_df['time'].min()

    for _, fixation in fixations_df.iterrows():
        start_time = fixation['time']
        duration = (fixation['duration'] / 1000.0) - time_offset  # Convert into seconds

        # Mark rows within the fixation region as 'Fixation' in both columns
        fixation_regions = (df['time'] >= start_time) & (df['time'] <= start_time + duration)
        df.loc[fixation_regions, 'movement_type'] = 'Fixation'
        df.loc[fixation_regions, 'smoothed_movement_type'] = 'Fixation'

predict_fixation(gaze_df, offset, time_offset=0.10, filepath=fixation_filepath)
fixation = gaze_df[(gaze_df['movement_type'] == 'Fixation') | (gaze_df['smoothed_movement_type'] == 'Fixation')]
print(f'Total Fixation Movements: {len(fixation)}')

Total Fixation Movements: 0


## Incorporating Annotations
Using the annotation.csv file, we want to generate new csv files that contains the following: object information, regions of observation, and experiment start/end

In [123]:
def analyse_annotations(annotations_csv_filepath, offset, fill_threshold=1):
    # This works through the the annotation and finds the following:
    # What object each data point is looking at
    # When an object spawns and intercepted

    # Group every spawn and interception into a single record (not all spawn may have an interception), could group by object ID?
    annotations_df = pd.read_csv(annotations_csv_filepath)
    
    # Format the timestamp into time relative to the experiment
    annotations_df['time'] = annotations_df['timestamp'] + offset
    annotations_df['time'] -= annotations_df['time'].min()

    # Filter only the data that contains 'Intercepted' or 'Spawning'.
    # Each object has their own spawning and interception time, could create records of these two
    # We can utilise this data as our own way to simplify the visualising and also finding TTC

    # Filter rows with 'Intercepted' or 'Spawning' label
    object_annotations = annotations_df[annotations_df['label'].isin(['Intercepted', 'Spawning'])]
    objects_df = object_annotations.pivot(index='id', columns='label', values='time')
    objects_df.reset_index(inplace=True)

    # If any object does not have both 'Spawning' and 'Intercepted' events, fill NaN with appropriate values
    objects_df.fillna({'spawning_timestamp': pd.NaT, 'intercepted_timestamp': pd.NaT}, inplace=True)

    # Now, create a new dataframe which stores when a user is looking at a specific region

    # Obtains all records with the 'Looking At' label
    looking_at_df = annotations_df[annotations_df['label'].isin(['Looking At'])]

    # Initialise an empty DataFrame to store regions
    regions_df = pd.DataFrame(columns=['id', 'start_time', 'end_time'])

    # Initialise variables for tracking consecutive points
    current_obj_id = None
    start_time = None

    # Iterate through each row in the 'Looking At' dataframe
    for _, row in looking_at_df.iterrows():
        obj_id = row['id']
        timestamp = row['time']

        # Check if it's the same object and within the threshold seconds
        if obj_id == current_obj_id and start_time is not None and timestamp - start_time <= fill_threshold: 
            end_time = timestamp # Update the end time for the current region
        else:
            # A new region is created when the next ID does not match the current region's ID OR if the time between two points is greater than the threshold (indicating it's a new region for the same ID)
            if current_obj_id is not None and start_time is not None:
                regions_df = pd.concat([regions_df, pd.DataFrame({'id': [current_obj_id], 'start_time': [start_time], 'end_time': [end_time]})], ignore_index=True)

            # Update tracking variables for the next iteration
            current_obj_id = obj_id
            start_time = timestamp
            end_time = timestamp

    # Add the last region after the loop
    if current_obj_id is not None and start_time is not None:
        regions_df = pd.concat([regions_df, pd.DataFrame({'id': [current_obj_id], 'start_time': [start_time], 'end_time': [end_time]})], ignore_index=True)

    # May want to include the Experiment Start and End lines as a JSON
    start_time = annotations_df.loc[(annotations_df['label'] == 'Experiment Started'), 'time'].values
    end_time = annotations_df.loc[(annotations_df['label'] == 'Experiment Ended'), 'time'].values

    experiment_info = {
        'Start': start_time.item(),
        'End': end_time.item()
    }

    return objects_df, regions_df, experiment_info

objects_df, regions_df, experiment_info = analyse_annotations(annotations_filepath, offset)

C:\Users\jhund\AppData\Local\Temp\ipykernel_4244\459282757.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  regions_df = pd.concat([regions_df, pd.DataFrame({'id': [current_obj_id], 'start_time': [start_time], 'end_time': [end_time]})], ignore_index=True)


### Finding TTC per interception
This function only works if we have incorporated the annotation as we need the interception that has been made. This iterates through each interceptiom find all the points that falls below the time that interception has been made and take the most recent saccadic movement to calculate the Time-To-Contact by taking the difference between


In [124]:
def find_TTC(gaze_df, objects_df):
    results = []

    for index, row in objects_df.iterrows():
        # Extract the 'Intercepted' time
        intercepted_time = row['Intercepted']
        
        # Filter gaze_df based on the condition where 'time' is less than the intercepted_time
        filtered_gaze_df = gaze_df[gaze_df['time'] < intercepted_time]

        # Obtain all the records where the movement type is Saccades
        smoothed_saccadic_gaze_df = filtered_gaze_df[filtered_gaze_df['smoothed_movement_type'] == "Saccades"]
        saccadic_gaze_df = filtered_gaze_df[filtered_gaze_df['movement_type'] == "Saccades"]

        # Obtains the latest saccadic movement before the intercepted time
        recent_saccade = saccadic_gaze_df['time'].max()
        smoothed_recent_saccade = smoothed_saccadic_gaze_df['time'].max()

        # Calculate the saccadic movement before the interception was made which is the difference between latest saccade and interception
        time_to_contact = intercepted_time - recent_saccade
        smooth_time_to_contact = intercepted_time - smoothed_recent_saccade
        results.append({
            'Object_ID': index+1, 
            'Interception': intercepted_time, 
            'Recent Saccade': recent_saccade, 
            'Time to Contact': time_to_contact,
            'Smoothed Recent Saccade': smoothed_recent_saccade,
            'Smoothed Time to Contact': smooth_time_to_contact})

    results_df = pd.DataFrame(results)
    return results_df

ttc_df = find_TTC(gaze_df, objects_df)

## Exporting Data
To be used for later or with the Visualier, this exports the dataframe and an json containing the offset for annotations

In [125]:
# Allows the dataframe to be exported as a csv file of a given name as well as a JSON file that stores the offset if the graph was to be used again
def export_gaze_data(gaze, objects, regions, ttc, experiment_info):
    
    output_directory = 'analysed_output'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    output_directory = 'visualise_source'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Saves a copy into the analysed output
    gaze.to_csv(f'analysed_output/gaze.csv', index=False)
    objects.to_csv(f'analysed_output/objects.csv', index=False)
    regions.to_csv(f'analysed_output/regions.csv', index=False)
    ttc.to_csv(f'analysed_output/ttc.csv', index=False)

    with open('analysed_output/experiment_info.json', 'w') as json_file:
        json.dump(experiment_info, json_file)

    # Saves another copy into the visualise_source 
    gaze.to_csv(f'visualise_source/gaze.csv', index=False)
    objects.to_csv(f'visualise_source/objects.csv', index=False)
    regions.to_csv(f'visualise_source/regions.csv', index=False)
    ttc.to_csv(f'visualise_source/ttc.csv', index=False)

    with open('visualise_source/experiment_info.json', 'w') as json_file:
        json.dump(experiment_info, json_file)

export_gaze_data(gaze_df, objects_df, regions_df, ttc_df, experiment_info) # Exporting the dataframe as a csv file